In [4]:
import pandas as pd
import numpy as np
import torch
import re
import string
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sentencepiece as spm
from typing import List
import joblib

d:\FYP2\Audio_model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
sp = spm.SentencePieceProcessor()
sp.Load("Tokenizer/Tanglish/taen_spm.model")

True

In [6]:
bilstm_model = load_model("./Classification/bilstm_model1.h5")

# Load BERT Model
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model_path = "ratish03/indic-BERT-Classification"
model.load_state_dict(torch.hub.load_state_dict_from_url(f"https://huggingface.co/{model_path}/resolve/main/best_tanglish_model.pt", map_location=torch.device('cpu')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

meta_model = joblib.load("Classification/meta_model.pkl")

def remove_single_characters(tokens: List[str]) -> List[str]:
    return [token for token in tokens if len(token) > 1]

def clean_text(text: str) -> str:
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

def lower_case_everything(t: str) -> str:
    return t.lower()

def replace_all_caps(tokens: List[str]) -> List[str]:
    return [f'xxup {t.lower()}' if t.isupper() else t for t in tokens]

def deal_caps(tokens: List[str]) -> List[str]:
    return [f'xxmaj {t}' if t.istitle() else t for t in tokens]

def handle_all_caps(t: str) -> str:
    tokens = t.split()
    tokens = replace_all_caps(tokens)
    return ' '.join(tokens)

def handle_upper_case_first_letter(t: str) -> str:
    tokens = t.split()
    tokens = deal_caps(tokens)
    return ' '.join(tokens)

custom_pre_rules = [lower_case_everything, handle_all_caps, handle_upper_case_first_letter]

def preprocess_text1(text: str) -> str:
    text = clean_text(str(text))
    for rule in custom_pre_rules:
        text = rule(text)
    return text

# Preprocessing function
def preprocess_text(text: str) -> str:
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text.lower()

# Tokenize with BiLSTM Tokenizer
class CodeMixedTanglishTokenizer:
    def __init__(self, model_path: str):
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(model_path)
    def tokenizer(self, items):
        return [self.sp.EncodeAsPieces(t) for t in items]

tanglish_tokenizer = CodeMixedTanglishTokenizer("./Tokenizer/Tanglish/taen_spm.model")

def predict_bilstm(text):
    test_text = [text]
    cleaned_text = [preprocess_text1(t) for t in test_text]
    tokenized = tanglish_tokenizer.tokenizer(cleaned_text)
    encoded = [sp.PieceToId(piece) for text in tokenized for piece in text]
    padded = pad_sequences([encoded], maxlen=70, padding="post")
    predictions = bilstm_model.predict(padded)
    confidence = np.max(predictions)
    label = np.argmax(predictions)
    return label, confidence

def predict_bert(text, model, tokenizer, device, max_len=128):
    model.eval()
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        prediction = torch.argmax(logits, dim=1).item()
    return prediction, probs[0][prediction].item()

def predict_category(bilstm_label, bilstm_confidence, bert_label, bert_confidence):
    input_data = np.array([[bilstm_label, bilstm_confidence, bert_label, bert_confidence]])
    prediction = meta_model.predict(input_data)
    return prediction[0]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
meta_model = load_model("Classification/meta_model_neural.h5")
scaler = joblib.load("Classification/scaler.pkl")

In [8]:
def predict_category1(bilstm_label, bilstm_confidence, bert_label, bert_confidence):
    input_data = np.array([[bilstm_label, bilstm_confidence, bert_label, bert_confidence]])
    input_data = scaler.transform(input_data)  # Normalize the input
    prediction = meta_model.predict(input_data)
    predicted_category = 1 if prediction[0] > 0.5 else 0  # Convert probability to class label
    return predicted_category

In [8]:
l = ["punda","mavane"]
for t in l:
    bilstm_label, bilstm_conf = predict_bilstm(t)
    bert_label, bert_conf = predict_bert(t,model,tokenizer,device)

    predicted_category = predict_category(bilstm_label,bilstm_conf,bert_label,bert_conf)
    print(f"{t} ---> {predicted_category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
punda ---> 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


d:\FYP2\Audio_model\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


mavane ---> 0


d:\FYP2\Audio_model\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [40]:
l = ["umae"]
for t in l:
    bilstm_label, bilstm_conf = predict_bilstm(t)
    bert_label, bert_conf = predict_bert(t,model,tokenizer,device)

    predicted_category = predict_category1(bilstm_label,bilstm_conf,bert_label,bert_conf)
    print(f"{t} ---> {predicted_category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
umae ---> 0


d:\FYP2\Audio_model\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
